# An Optimized Code for Conway's Game of Life

### 1. Introduction

In 1970, British mathematician John Horton Conway introduced his cellular automaton called Game of Life. A cellular automaton simply consists of a grid of cells which can either be alive or dead. They interact with neighboring cells according to some fixed rules which make the system evolve each timestep. Game of Life is not only a fruitful playground for theoretical computer scientists but also it is fun to watch the cells exhibit interesting collective behaviours. Sometimes, they oscillate between different shapes or another time you can find several cells form a spaceship and cruise on the grid.

The problem has been extensively studied throughout the years. One can find many codes to simulate it. However, when I took a look at the codes on popular websites such as Youtube or Medium, I found that most of them have been using nested iterations which makes their simulations run very slow. What makes my code different is that it is vectorized and branchless. I will cover these topics on another article in depth. Here I will briefly mention what they are and focus mostly on implementation. I believe that this is a good practice for seeing these consepts in action. Throughout this article, I will show you step by step how I implemented my code from scratch. I will also implement the slow version so that we can compare the speeds.

### 2. Rules of the Game

Consider nxn square grid. Each cell can either be in dead or alive state. We assign 0 for dead and 1 for alive. This was the rule for cells. A cell has 8 neighbors, which are the cells that are horizontally, vertically, or diagonally adjacent. Now, introduce four other rules about how their interactions take place with neighboring cells. Each of these four rules mimics a certain behaviour.

__1)__ If alive cell has fewer than two alive neighbours, then it dies. (underpopulation)

__2)__ If alive cell has two or three alive neighbours, then it stays alive.

__3)__ If alive cell has more than three alive neighbours, then it dies. (overpopulation)

__4)__ If dead cell has exactly three alive neighbours, then it becomes an alive cell. (reproduction)

Actually, the first three rules can be abbreviated as "if alive cell does not have two or three alive neighbours, then it dies".

### 3. The structure of the Code

The general structure of the code is as follows:

In [ ]:
class GameofLife:
    def __init__():
        # initilize the system and relevant parameters
        
    def _step_unoptimized():
        # calculate a step and update the system using the unoptimized algorithm
    
    def _step_optimized():
        # calculate a step and update the system using the optimized algorithm
        
    def run_and_plot():
        # evaluate the chosen step function and plot it

### 4. Implementing the Code

We first start by importing required libraries. As we proceed, you will see why we have imported these libraries.

In [ ]:
import numpy as np
from time import time, sleep
from scipy.signal import convolve2d
from matplotlib import pyplot as plt

Now, define a class called GameofLife and initilize with input parameters dim, max_step, seed and sparseness.

In [ ]:
class GameofLife:
    def __init__(self, dim, max_step, seed, sparseness=0.5):

        self.dim = dim  # dimension along one direction
        self.max_step = max_step  # maximum number of timestep

        # must be in (0, 1). 0.5 = equal distribution. the closer to 0, the emptier the system
        self.sparseness = sparseness

        # initiate a random board
        np.random.seed(seed)
        self.board = np.random.choice([0, 1], p=[self.sparseness, 1-self.sparseness], size=[self.dim, self.dim])

__dim__ is the dimension of the system along one direction. For example, if dim=10, then there will be dim*dim = 100 cells in the system. 

__max_step__ determines the maximum number of timestep. I will explain sparseness in a minute. 

__np.random.seed(seed)__ is, as its name suggests, the seed of the random number generator.

__self.board__ is the grid where we store the information of the cell states. It is a self.dim x self.dim matrix and initilized using np.random.choice function. The size parameter determines the dimensions. First parameter, [0, 1], shows that from which numbers are sampled. And, p parameter shows the sampling distribution for each numbers from the list given in the first parameter. For example, if p=[0.5, 0.5], then the probability of having 0 or 1 will be equal. If p=[0, 1], then the result will always be 1. __sparseness__ variable controls this distribution. It basically determines how many alive cells there are 
at the beginning. If sparseness = 1, then all the cells starts dead therefore there is no alive cell and the system looks very sparse. That is why I named it like that. On the other hand, for small sparseness values, the number of alive cell gets higher.

In [ ]:
    def _step_unoptimized(self):

        temp_board = np.zeros((self.dim, self.dim))

        for i in range(self.board.shape[0]):
            for j in range(self.board.shape[1]):
                temp_sum = self.board[(i+1) % self.dim, j % self.dim] \
                           + self.board[i % self.dim, (j+1) % self.dim] \
                           + self.board[(i-1) % self.dim, j % self.dim] \
                           + self.board[i, (j-1) % self.dim] \
                           + self.board[(i+1) % self.dim, (j+1) % self.dim]\
                           + self.board[(i-1) % self.dim, (j+1) % self.dim]\
                           + self.board[(i-1) % self.dim, (j-1) % self.dim]\
                           + self.board[(i+1) % self.dim, (j-1) % self.dim]

                if self.board[i, j] == 1:
                    if temp_sum < 2 or temp_sum > 3:
                        temp_board[i, j] = 0
                    else:
                        temp_board[i, j] = 1

                elif self.board[i, j] == 0:
                    if temp_sum == 3:
                        temp_board[i, j] = 1

        self.board = temp_board

The implementation of the unoptimized step is straightforward. We first create an empty grid called ```temp_board``` which will be used temporarily for each iteration. Then, we simply iterate through all the cells using the nested iteration. For each cell, we calculate the number of neighboring alive cells by summing the values. One should be careful with the cell on the boundaries since they have less than 8 neighbors. Here, we use periodic boundary conditions. The mod operator % takes care of the system boundaries and connect the edges at the opposite sides. The topology of the system becomes equivalent to a torus. 

Let us now implement the optimized version. To do that, we first make the code vectorized. Vectorization is simply writing a code by getting rid of explicit for-loops. Vectorization takes the advantage from the parallelization (parallelization instruction called SIMD in CPUs and GPUs to be more spesific) and thus it significantly improves the performance. I will cover the concept in more detail in another article.

```temp_sum``` in ```_step_unoptimized``` method calculates the sum of the values in neighboring cells. This can be done by using convolution in one go. If we choose a convolution kernel as follows

$K=\left( {\begin{array}{ccc}
   1 & 1 & 1\\
   1 & 0 & 1\\
   1 & 1 & 1\\
  \end{array} } \right)$
  
then it will calculate the sum of the surrounding cells and write to a new array, say ```temp_board```. ```temp_board``` contains the information of how many alive neighboring cell there are. Here, we also have the same problem at the boundaries. It can easily be solved by concatenating the columns or rows of opposing edges of ```temp_board``` to each other. Numpy has this as a built-in feature in ```convolve2d``` method with the keyword argument ```boundary="wrap"```.

In [ ]:
    def _step_optimized(self):
        conv_kernel = np.array([[1, 1, 1],
                                [1, 0, 1],
                                [1, 1, 1]])

        temp_board = convolve2d(self.board, conv_kernel, mode="same", boundary="wrap")

We now need to decide which cells will stay alive or dead in the next timestep. We can naively iterate through ```temp_board``` and make inquiry for each cell. This would be enormously slow because of two reasons. First, it is not vectorized and second in each iteration if-statement divides the code into branches. If there were no branch, then CPU simply executes the instructions and will be prepared for the upcoming instructions. In case of branching, CPU cannot know which direction the code will take so it does not know which instruction to load. If it is directed to a wrong branch, then it needs to flush all the instruction it loaded. This may take a lot of time. In branchless coding, we try to get rid of or, at least, minimize the number of if-statements. Here, we handle the issue by multiplying temp_broad by appropriate boolean matrices. The code piece that does the job can be written as follows

In [ ]:
    self.board = np.logical_or(np.logical_and((self.board == 1), np.logical_or(temp_board == 2, temp_board == 3)),
                                       np.logical_and((self.board == 0), (temp_board == 3)))

At first glance, it does not look meaningful so it is better to look at the code in pieces. Let us consider the part  ``` np.logical_and((self.board == 0), (temp_board == 3)) ```.  

```(self.board == 0)``` returns a boolean array where ```self.board``` takes 0 (dead) values. Similarly,  ```(temp_board == 3)``` returns the cells that only has 3 alive neighbor. By the 4th rule of the game, this cell must be alive in the next timestep. ```np.logical_and()``` function is logical AND operator for matrices. 

By the same logic, ```np.logical_and((self.board == 1), np.logical_or(temp_board == 2, temp_board == 3)``` takes care of the first three rules. In the end, we combine the conditions with ```np.logical_or()```. This ends the implementation of the optimized part.

All we need to do is to plot a given matrix. The following code takes a matrix, in our case it is ```temp.board```, and plot the matrix monochromatically. It also evaluate the system with specified scheme (```optimized=True``` for ```self._step_optimized()``` and ```optimized=False``` for ```self._step_unoptimized()```). One special thing about the plotting part is that it uses blitting. Blitting drastically increase the plotting speed because it avoids to render stationary object (such as axes, plot title, legends etc.) multiple times.

In [ ]:
    def run_and_plot(self, optimized=True, pause_between_frames=0.05):
        fig, ax = plt.subplots()
        ln = ax.imshow(self.board, animated=True, cmap="binary")
        plt.show(block=False)
        plt.pause(0.1)
        bg = fig.canvas.copy_from_bbox(fig.bbox)
        ax.draw_artist(ln)
        fig.canvas.blit(fig.bbox)

        for j in range(self.max_step):
            if optimized:
                self._step_optimized()
            else:
                self._step_unoptimized()

            fig.canvas.restore_region(bg)
            ln = ax.imshow(self.board, animated=True, cmap="binary")
            ax.draw_artist(ln)
            fig.canvas.blit(fig.bbox)
            fig.canvas.flush_events()
            sleep(pause_between_frames)

We can now test the speed of both implementations. I have a computer with Intel i7-6700k processor and 16 GB of RAM.

For dim=150, max_step=25, sparseness=0.5, seed=42:

- Evaluation time of optimized method: 0.7210686206817627 s
- Evaluation time of unoptimized method: 2.8753082752227783 s

For dim=250, max_step=25, sparseness=0.5, seed=42:

- Evaluation time of optimized method: 0.7220661640167236 s
- Evaluation time of unoptimized method: 5.3498430252075195 s

For dim=500, max_step=25, sparseness=0.5, seed=42:

- Evaluation time of optimized method: 0.9230928421020508 s
- Evaluation time of unoptimized method: 19.079766988754272 s

I checked the speeds for different dimensions (dim=150, 250, 500). It can clearly be seen that our optimized method is significantly faster. For example, for dim=500, it is rougly 20 times faster. The order does not go linearly with the dimension. It means that, as the dimension grows, the difference in the speed gets higher.